In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import gkey
gmaps.configure(api_key = gkey)

In [2]:
city_df = pd.read_csv("city.csv")
print(f"number of records: {len(city_df)}")
city_df.dropna(how = "any", inplace = True)
print(f"number of records after removal of nulls: {len(city_df)}")

number of records: 600
number of records after removal of nulls: 541


In [3]:
locations = city_df[["Latitude", "Longitude"]]
weights = city_df["Humidity"]

In [4]:
fig = gmaps.figure(center = (0,0), zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights = weights, dissipating = False, max_intensity = 200, point_radius = 5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
hotel_df = city_df.loc[(city_df["Max Temperature"] >= 23) & (city_df["Max Temperature"] < 33) & (city_df["Humidity"] < 35) & (city_df["Wind Speed"] < 15), :]
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Lat"] = ""
hotel_df["Hotel Lng"] = ""
hotel_df

<ipython-input-5-90ee5aa89e22>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""
<ipython-input-5-90ee5aa89e22>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Lat"] = ""
<ipython-input-5-90ee5aa89e22>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

,City,Max Temperature,Humidity,Wind Speed,Cloudiness,Latitude,Longitude,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
0,busselton,31.14,29.0,1.96,0.0,-33.6500,115.3333,AU,1.643345e+09,,,
120,vaini,27.27,31.0,10.61,32.0,-43.3002,-65.1023,AR,1.643345e+09,,,
134,nanortalik,24.22,30.0,7.77,0.0,-39.0333,-67.5833,AR,1.643345e+09,,,


In [6]:
radius = 5000
target_search = "hotel"

In [7]:
# params dictionary to update each iteration
params = {
    "radius": radius,
    "types": "lodging",
    "keyword": target_search,
    "key": gkey
}

# Use the lat/lng pulled in for loop
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel", using our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and convert into json, using pprint to enable data structure
    hotel_address = requests.get(base_url, params=params)
    hotel_address = hotel_address.json()
    #pprint(hotel_address)
    
    # leverage try/except in case data element empty. Use pass to skip exception.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Lat"] = hotel_address["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Hotel Lng"] = hotel_address["results"][0]["geometry"]["location"]["lng"]
        # print updated df with hotel name to validate df updated properly
        #print(hotel_df["Hotel Name"], hotel_address["results"][0]["name"])
    except (KeyError, IndexError):
        pass
    
print(hotel_df)

C:\Users\freed\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


           City  Max Temperature  Humidity  Wind Speed  Cloudiness  Latitude  \
0     busselton            31.14      29.0        1.96         0.0  -33.6500   
120       vaini            27.27      31.0       10.61        32.0  -43.3002   
134  nanortalik            24.22      30.0        7.77         0.0  -39.0333   

     Longitude Country          Date                Hotel Name  Hotel Lat  \
0     115.3333      AU  1.643345e+09  Bayview Geographe Resort -33.658535   
120   -65.1023      AR  1.643345e+09         Hosteria Sampedro -43.301509   
134   -67.5833      AR  1.643345e+09                Hotel Bait -39.041341   

     Hotel Lng  
0    115.27671  
120 -65.104922  
134 -67.587945  


In [8]:
fig = gmaps.figure()

